 We implement surface sensing by varying k_ret_on
 to create a delayed retraction after contacting the surface

In [1]:
import os, sys
join = os.path.join
import numpy as np
import pandas as pd
import stats
import json
import matplotlib.pyplot as plt
from tabulate import tabulate
import SALib.analyze as sal
#
import rtw
import _fj
import txtdata
import sobol
import twanalyse
import twutils

from sobol import collect_obs, compute_sobol, format_sobol

In [2]:
# start with sobol at 
notedir, notename = os.path.split(os.getcwd())
simdir = "/home/dan/usb_twitching/run/b2392cf/cluster/sobol_ssense"
lookup = sobol.read_lookup(simdir)
problem = sobol.read_problem(simdir)
twutils.print_dict(problem)


{
	"num_vars": 6,
	"names": [
		"k_ext_off",
		"dwell_time",
		"pilivar",
		"anchor_angle_smoothing_fraction",
		"k_spawn",
		"k_ret_on"
	],
	"bounds": [
		[
			0.2,
			1.0
		],
		[
			0.5,
			3.0
		],
		[
			1.0,
			20.0
		],
		[
			0.125,
			1.0
		],
		[
			0.5,
			5.0
		],
		[
			0.25,
			2.5
		]
	]
}


In [3]:
# first job is to check that the simulation and analysis actually ran
stats_mask = np.array([os.path.exists(join(simdir, udir, "local.json")) for udir in lookup[0]])
num, denom = np.count_nonzero(stats_mask), len(lookup[0])
print('ran samples {}/{} ({:.1f}%)'.format(num, denom, 100*float(num)/denom))


ran samples 8192/8192 (100.0%)


In [4]:
# load summary statistics 
objectives = ['lvel.mean', 'deviation.var', 'qhat.estimate', 'ahat.estimate', 
    'nbound.mean', 'ntaut.mean']
Y, lduid = sobol.collect(objectives, targetdir=simdir, alldata=True)

In [5]:
# check for missing data
missing = sobol.check_missing(lookup, Y)
if missing:
    print(tabulate(missing, headers=["objective", "nan data"]))
else:
    print("No nan data")


No nan data


In [6]:
import seaborn as sns

# sns.histplot(Y["lvel.mean"])
# sns.histplot(Y["deviation.var"])
# sns.histplot(Y["qhat.estimate"])


In [7]:

second_order = False
Si = sobol.compute_sobol(problem, Y, second_order=second_order)
Si["lvel.mean"]["ST"]

dftable1, dftableT = sobol.format_sobol(problem, Si)


In [8]:
from IPython.display import display, HTML
display(HTML(dftable1.to_html()))
display(HTML(dftableT.to_html()))

with open("/home/dan/usb_twitching/notes/sensitivity/tex/ssense_table.tex", "w") as f:
    f.write(dftableT.to_latex())



,parameter,lvel.mean,deviation.var,qhat.estimate,ahat.estimate,nbound.mean,ntaut.mean
0,k_ext_off,0.198742,0.001493,0.074316,0.000081,0.177328,0.076199
1,dwell_time,0.089151,0.053561,0.031144,-0.004856,0.134489,0.240522
2,pilivar,0.012466,0.742659,0.280714,0.028291,0.019250,0.139748
3,anchor_angle_smoothing_fraction,0.263004,0.083201,0.272238,0.911922,0.000096,0.034144
4,k_spawn,0.320414,0.040521,-0.026464,-0.005645,0.241867,0.203588
5,k_ret_on,0.003083,0.002703,-0.028310,-0.003490,0.183608,0.005897


,parameter,lvel.mean,deviation.var,qhat.estimate,ahat.estimate,nbound.mean,ntaut.mean
0,k_ext_off,0.269460,0.029726,0.199713,0.030268,0.266198,0.134813
1,dwell_time,0.128732,0.091215,0.224852,0.029655,0.238807,0.409065
2,pilivar,0.074694,0.779365,0.432965,0.067084,0.028247,0.238254
3,anchor_angle_smoothing_fraction,0.339103,0.124867,0.386952,0.951388,0.003808,0.086274
4,k_spawn,0.384593,0.073556,0.126572,0.027904,0.362955,0.343865
5,k_ret_on,0.023590,0.014938,0.089180,0.020065,0.316452,0.037154


 We see that the retraction delay (k_ret_on)
 has almost no effect on the dynamics. Despite influencing nbound.mean
 it does not change ntaut.mean. We do not allow the extension
 motor to force pili into bent configurations but we do allow
 "bent" (l_ideal < l_contour) configurations due to attachment
 and changing body position.
 We expect pili which are not retracting to rarely be involvd in the dynamics
 This may not be the case if the bacteria has low or negative persistence
 e.g. In walking state.
 TODO redo this simulation in walking state